In [4]:
import imaplib
import email

In [8]:
mail_adr = 'magal201121@gmail.com'
mail_ps = 'goyq gwyj nmpr nigo'

In [6]:
imap = imaplib.IMAP4_SSL('smtp.gmail.com', "993")

In [9]:
imap.login(mail_adr, mail_ps)

error: b'[AUTHENTICATIONFAILED] Invalid credentials (Failure)'

In [11]:
import imapclient
import getpass

In [12]:
password = getpass.getpass("Enter your Gmail password: ")

In [13]:
password

'dhdmseka'

In [14]:
input()

'dhdmseka'

In [16]:
type(password)

str

In [23]:
EMAIL = 'mafal201121@gmail.com'  # 본인의 이메일 주소를 입력하세요.

# 암호를 입력합니다. (입력 시 암호가 표시되지 않습니다)
password = 'goyqgwyjnmprnigo'

# IMAP 서버에 연결하고 로그인합니다
imap_obj = imapclient.IMAPClient('imap.gmail.com', ssl=True)
imap_obj.login(EMAIL, password)


b'mafal201121@gmail.com authenticated (Success)'

In [24]:
# 'INBOX' 폴더를 선택합니다
imap_obj.select_folder('INBOX', readonly=True)


{b'PERMANENTFLAGS': (),
 b'FLAGS': (b'\\Answered',
  b'\\Flagged',
  b'\\Draft',
  b'\\Deleted',
  b'\\Seen',
  b'$NotPhishing',
  b'$Phishing'),
 b'UIDVALIDITY': 1,
 b'EXISTS': 940,
 b'RECENT': 0,
 b'UIDNEXT': 1063,
 b'HIGHESTMODSEQ': 315570,
 b'READ-ONLY': [b'']}

In [26]:
# 최근 10개의 메일을 검색합니다
email_ids = imap_obj.search(['ALL'])[-10:]

In [29]:
# 메일 본문을 가져옵니다
for email_id in email_ids:
    raw_message = imap_obj.fetch([email_id], ['BODY[]', 'FLAGS'])
    print(raw_message[email_id][b'BODY[]'].decode('utf-8'))



error: command UID illegal in state LOGOUT, only allowed in states SELECTED

In [28]:
imap_obj.logout()

b'LOGOUT Requested'